In [2]:
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [32]:
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
from torch.nn import BCEWithLogitsLoss
from transformers import TFAutoModel, AutoTokenizer
from transformers import DistilBertForSequenceClassification
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import string
import torch
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/IMDBText

Mounted at /content/drive
/content/drive/MyDrive/IMDBText


In [4]:
import pandas as pd
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Preprocessing

In [5]:
# Function for text preprocessing
def preprocess_text(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.lower() not in stop_words and token not in string.punctuation]
    return ' '.join(tokens)

In [6]:
# Preprocess the text data
train_data['review'] = train_data['review'].apply(preprocess_text)
test_data['review'] = test_data['review'].apply(preprocess_text)


In [7]:
train_data['sentiment'] = train_data['sentiment'].map({'negative': 0, 'positive': 1})
test_data['sentiment'] = test_data['sentiment'].map({'negative': 0, 'positive': 1})

In [8]:
train_data.head()

,review,sentiment
0,SAPS SEA br br Aspect ratio 1.37:1 br br Sound...,0
1,want mindless action hot chick post-apocalypti...,1
2,`` Woman Black '' easily one creepiest British...,1
3,barely find word describe much piece trash off...,0
4,'s Let tell 's presence Alec Baldwin 's great ...,0


In [9]:
train_texts, train_labels = train_data['review'], train_data['sentiment']
test_texts, test_labels = test_data['review'], test_data['sentiment']

In [10]:
train_texts.reset_index(drop=True, inplace=True)
test_texts.reset_index(drop=True, inplace=True)

In [11]:
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

train_texts.reset_index(drop=True, inplace=True)
val_texts.reset_index(drop=True, inplace=True)
train_labels.reset_index(drop=True, inplace=True)
val_labels.reset_index(drop=True, inplace=True)

In [16]:
import torch

class IMDB(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

        # Ensure all encodings and labels have consistent lengths
        assert all(len(val) == len(self.labels) for val in self.encodings.values()), "Encodings and labels lengths do not match."

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

In [13]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("roberta-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True)

In [19]:
train_data = IMDB(train_encodings, train_labels)
test_data = IMDB(test_encodings,test_labels)
val_data = IMDB(val_encodings, val_labels)

In [20]:
from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, AdamW

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device=device)
model.train()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [21]:
train_dataloader = DataLoader(dataset=train_data, batch_size=16, shuffle=True)
val_dataloader = DataLoader(dataset=val_data, batch_size=16, shuffle=False)
test_dataloader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)

In [22]:
optim = AdamW(model.parameters(),lr=1e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [23]:
total_start_time = time.time()  # Start timing for the total training time


for epoch in range(3):
    epoch_start_time = time.time()  # Start timing for the epoch

    model.train()  # Set the model to training mode
    train_loss = 0
    for batch in tqdm(train_dataloader):
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()
        train_loss += loss.item()
    train_loss /= len(train_dataloader)  # Calculate the average loss over all training batches

    # Validation phase
    model.eval()  # Set the model to evaluation mode
    val_loss = 0
    with torch.no_grad():  # Do not compute gradient to speed up computation and reduce memory usage
        for batch in val_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs[0]
            val_loss += loss.item()
    val_loss /= len(val_dataloader)  # Calculate the average loss over all validation batches

    epoch_end_time = time.time()  # End timing for the epoch


100%|██████████| 1500/1500 [37:26<00:00,  1.50s/it]


In [25]:
# Print the losses and the time taken for the epoch
print(f"Epoch {epoch+1}/{3}: Train loss = {train_loss:.4f}, Validation loss = {val_loss:.4f}")
print(f"Time taken for epoch {epoch+1}: {epoch_end_time - epoch_start_time:.2f} seconds")

total_end_time = time.time()  # End timing for the total training time
print(f"Total training time: {total_end_time - total_start_time:.2f} seconds")

Epoch 3/3: Train loss = 0.1187, Validation loss = 0.2255
Time taken for epoch 3: 2441.30 seconds
Total training time: 7390.53 seconds


***Saving the model into Drive***



In [26]:
saved_model = "/content/drive/MyDrive/IMDBText"

In [28]:
tokenizer.save_pretrained(saved_model)
model.save_pretrained(saved_model)

***Uploading the Saved Model***

In [30]:
tokenizer = AutoTokenizer.from_pretrained(saved_model)
model = DistilBertForSequenceClassification.from_pretrained(saved_model, return_dict=False)

You are using a model of type roberta to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/IMDBText and are newly initialized: ['classifier.bias', 'classifier.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.word_embeddings.weight', 'pre_classifier.bias', 'pre_classifier.weight', 'transformer.layer.0.attention.k_lin.bias', 'transformer.layer.0.attention.k_lin.weight', 'transformer.layer.0.attention.out_lin.bias', 'transformer.layer.0.attention.out_lin.weight', 'transformer.layer.0.attention.q_lin.bias', 'transformer.layer.0.attention.q_lin.weight', 'transformer.layer.0.attention.v_lin.bias', 'transformer.layer.0.attention.v_lin.weight', 'transformer.layer.0.ffn.lin1.bias', 'transformer.layer.0.ffn.lin1.weight', 'transformer.l

In [31]:
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-11): 12 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)

In [33]:
model.eval()  # Ensure the model is in evaluation mode

# Store predictions and actual labels
predictions = []
actuals = []

with torch.no_grad():
    for batch in tqdm(test_dataloader):
        # Move tensors to the same device as the model
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)  # Only needed if you're also evaluating performance

        outputs = model(input_ids, attention_mask=attention_mask)

        # Assuming you're doing classification and want the highest probability class
        logits = outputs[0]
        predicted_labels = torch.argmax(logits, dim=1)
        predictions.extend(predicted_labels.cpu().numpy())

100%|██████████| 20000/20000 [12:46<00:00, 26.09it/s]


In [34]:
actual_labels = []
with torch.no_grad():
    for batch in tqdm(test_dataloader):
        labels = batch['labels'].to(device)  # Assuming labels are on the same device
        actual_labels.extend(labels.cpu().numpy())

100%|██████████| 20000/20000 [00:12<00:00, 1592.01it/s]


In [35]:
correct_predictions = sum(p == a for p, a in zip(predictions, actual_labels))
accuracy = correct_predictions / len(predictions)
print(f"RoBERTa Accuracy on Test Data: {accuracy * 100:.2f}%")

RoBERTa Accuracy on Test Data: 49.15%
